<a href="https://colab.research.google.com/github/jmbaek/DNABERT/blob/master/DNABERT_run.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [17]:
!git clone https://github.com/jerryji1993/DNABERT

Cloning into 'DNABERT'...
remote: Enumerating objects: 766, done.
remote: Counting objects: 100% (766/766), done.
remote: Compressing objects: 100% (529/529), done.
remote: Total 766 (delta 400), reused 552 (delta 214), pack-reused 0
Receiving objects: 100% (766/766), 11.78 MiB | 3.90 MiB/s, done.
Resolving deltas: 100% (400/400), done.


In [26]:
%cd /content/DNABERT
!pwd

/content/DNABERT
/content/DNABERT


In [82]:
!python -m pip install --editable .
!cd examples
!python3 -m pip install -r requirements.txt

ERROR: File "setup.py" or "setup.cfg" not found. Directory cannot be installed in editable mode: /content/t
/bin/bash: line 0: cd: examples: No such file or directory
ERROR: Could not open requirements file: [Errno 2] No such file or directory: 'requirements.txt'


In [81]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
https://drive.google.com/file/d/1BJjqb5Dl2lNMg2warsFQ0-Xvn1xxfFXC/view?usp=sharing

In [88]:
from zipfile import ZipFile
file_name = "/content/t/6-new-12w-0.zip"

with ZipFile(file_name, 'r') as zip:
  zip.extractall()
  print('Done')

BadZipFile: ignored

In [69]:
!mv /6-new-12w-0.zip /content/t/6-new-12w-0.zip

mv: cannot stat '/6-new-12w-0.zip': No such file or directory


In [80]:
!unzip -Z /content/t/6-new-12w-0.zip


Archive:  /content/t/6-new-12w-0.zip
[/content/t/6-new-12w-0.zip]
  End-of-central-directory signature not found.  Either this file is not
  a zipfile, or it constitutes one disk of a multi-part archive.  In the
  latter case the central directory and zipfile comment will be found on
  the last disk(s) of this archive.
zipinfo:  cannot find zipfile directory in one of /content/t/6-new-12w-0.zip or
          /content/t/6-new-12w-0.zip.zip, and cannot find /content/t/6-new-12w-0.zip.ZIP, period.


In [50]:
!python3 /content/DNABERT/examples/run_finetune.py --data_dir /content/DNABERT/examples/sample_data/ft/6/ \
                        --output_dir /content/ --model_name_or_path /content/t/ --task_name T --model_type dna --visualize_train --do_train 

Traceback (most recent call last):
  File "/content/DNABERT/examples/run_finetune.py", line 1284, in <module>
    main()
  File "/content/DNABERT/examples/run_finetune.py", line 1000, in main
    args.output_dir
ValueError: Output directory (/content/) already exists and is not empty. Use --overwrite_output_dir to overcome.
